I am trying to convince a person who has no clue about data science which is the best way to release his movie,my conclusion has to be easily understandable by him,thats the difficult part in data science.

In [38]:
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import spacy
import pickle 
import spacy
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()
import spacy
import  pickle
from gensim.corpora import Dictionary
from tqdm import tqdm_notebook
import gensim
from gensim.models import Word2Vec
import re


I presend my steps through this procedure,firstly i split the dataset into 2 subsets.Assuming is the best way to realise which is the best way to approach this problem.I beging preprocessing my data with spacy,split into tokens,remove stop words, lemmatize and remove special characters if i see fit.Second i try to create both bag of words(BOW) i use Word2Vec for that with size=300 to have a good embedding size and i only take words with above 5 appearences.Creating a list out of the dictionary with the most frequent and unique word for every subset.Now that i have the BOW for both linear and netflix we can create a list of words.I named it satisfied list, because at the end of day what matters most is to find which type will engage the audience more.Then i create a log-likelihood measure to compare the the frequency of BOWs between the list and the one with the biggest score will be the case for our analysis.Like if the BOW from netflix has better score to the satisfied list then my conclusion to the producer is to release his movie with the type of netflix otherwise with the type of linear

In [39]:
df = pd.read_pickle('discussions_corrected.p')

In [40]:
df.head()

title  ...                                               post
0  Better Call Saul  ...                  Walter. And there the chain ends.
1  Better Call Saul  ...  I love this show. But it's hard to argue again...
2  Better Call Saul  ...  What am I missing?  A lot of reference to ribs...
3  Better Call Saul  ...          Oh come on Mike, he's a good little boy. 
4  Better Call Saul  ...                                       Look again 👀

[5 rows x 4 columns]

In [41]:
df.title.value_counts()

Game of Thrones            15462
Breaking Bad                6424
Better Call Saul            5268
Black Mirror                4720
Stranger Things             2891
True Detective              2721
Twin Peaks                  2399
Dark                        2004
Ozark                       1417
Mr. Robot                   1347
Orange is the New Black     1208
The Witcher                 1188
Fargo                        680
Mindhunter                   657
The Newsroom                 484
Succession                   389
The Crown                    338
House of Cards               158
La Casa de Papel             150
The Mandelorian               95
Name: title, dtype: int64

In [43]:
#spliting the dataset into 2 subsets through type
df_linear = df[df.type == 'linear']
df_netflix = df[df.type == 'netflix']


In [44]:
nlp = spacy.load("en_core_web_sm")

In [45]:
# use the pipe to preprocess the data and tokenize them
processed_texts = [text for text in tqdm_notebook(nlp.pipe(df_linear['post'], 
                                              n_process=-1, # maximum number of threads
                                              disable=["ner",
                                                       "parser"]),
                                         total=len(df_linear['post']))]



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [46]:
tokenized_df_linear = [[token.lemma_ for token in text
               if not token.is_punct and not token.is_stop]
              for text in processed_texts]                                         


In [47]:
 tokenized_df_linear = [[re.sub(r'\W+', '', word) for word in text] for text in tokenized_df_linear] #we remove the words we dont want
 tokenized_df_linear[:1]

[['Walter', 'chain', 'end']]

In [48]:
 tokenized_df_linear2 = [[re.sub(r'\b\w{1,3}\b', '', word) for word in text] for text in tokenized_df_linear] # we remove the words that are 3 or more,in my opinion they are noise
 tokenized_df_linear2[:1]


[['Walter', 'chain', '']]

In [49]:
processed_texts = [text for text in tqdm_notebook(nlp.pipe(df_netflix['post'], 
                                              n_process=-1, # maximum number of threads
                                              disable=["ner",
                                                       "parser"]),
                                         total=len(df_netflix['post']))]


tokenized_df_netflix = [[token.lemma_ for token in text
               if not token.is_punct and not token.is_stop]
              for text in processed_texts]                                        


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [50]:
tokenized_df_netflix = [[re.sub(r'\W+', '', word) for word in text] for text in tokenized_df_netflix] #remove the \n
tokenized_df_netflix[:1]

[['think', 'rabbit', 'end', 'real', 'ending', 'traincrasch']]

In [51]:
tokenized_df_netflix2 = [[re.sub(r'\b\w{1,3}\b', '', word) for word in text] for text in tokenized_df_netflix] #remove the numbers
tokenized_df_netflix2[:2]


[['think', 'rabbit', '', 'real', 'ending', 'traincrasch'],
 ['wipe', '', 'tracker', 'realize', 'fate']]

In [52]:
tokenized_df_linear[:1]


[['Walter', 'chain', 'end']]

In [53]:
tokenized_df_netflix[:2]

[['think', 'rabbit', 'end', 'real', 'ending', 'traincrasch'],
 ['wipe', 'eat', 'tracker', 'realize', 'fate']]

In [54]:
#i use W2v to create the WOB 
SIZE = 300 # dimensions of the embeddings
SG = 1 # whether to use skip-gram or CBOW (we use skip-gram)
WINDOW = 10 # the window size
N_WORKERS = 1 # number of workers to use
MIN_COUNT = 5

model = Word2Vec(size=SIZE,
                sg=SG,
                window=WINDOW, 
                min_count=MIN_COUNT,
                workers=N_WORKERS)

model.build_vocab(tokenized_df_netflix2)

model.train(tokenized_df_netflix2,
           total_examples=model.corpus_count,
           epochs=model.epochs)

(675332, 997445)

In [55]:
len(model.wv.vocab)

3897

In [56]:
# i create the first wob from the netflix and they are all unique words 
BOW1 = []
for key, value in model.wv.vocab.items(): 
  BOW1.append(key)

    


In [57]:
WOB1[:2] #now the datas are clean 

['think', 'rabbit']

In [58]:
SIZE = 300 # dimensions of the embeddings
SG = 1 # whether to use skip-gram or CBOW (we use skip-gram)
WINDOW = 10 # the window size
N_WORKERS = 1 # number of workers to use
MIN_COUNT = 5

model = Word2Vec(size=SIZE,
                sg=SG,
                window=WINDOW, 
                min_count=MIN_COUNT,
                workers=N_WORKERS)

model.build_vocab(tokenized_df_linear2)

model.train(tokenized_df_linear2,
           total_examples=model.corpus_count,
           epochs=model.epochs)

(1220047, 1771970)

In [59]:
# i create the first wob from the linear and they are all unique words 
BOW2 = []
for key, value in model.wv.vocab.items(): 
  BOW2.append(key)

In [60]:
BOW2[:2]

['Walter', 'chain']

Now that i have the BOW for both linear and netflix i can create a new list of words,which will contain words that declare good ratio.I named it satisfied list.Then i create a log-likelihood measure to compare the the frequency of BOWs between the list and the one with the biggest score will be the case for our analysis.

In [61]:
sarisfied_list = ['good','nice','gratified','happy','joyful','joyous']#i could go on but i think the idea is the point here not the actual result

In [80]:
from collections import Counter

flatten = lambda t: [item for sublist in t for item in sublist]

In [81]:
# i will not forget to flatten
flatten_WOB1 = flatten(BOW1)
flatten_WOB2 = flatten(BOW2)


In [91]:
from collections import Counter
from scipy.stats import chi2_contingency

tokenized_text_corpus_1 = BOW1
tokenized_text_corpus_2 = sarisfied_list

counts_c1 = Counter(tokenized_text_corpus_1)
counts_c2 = Counter(tokenized_text_corpus_2)

freq_c1_other = sum(counts_c1.values()) 
freq_c2_other = sum(counts_c2.values())

In [92]:


llr, p_value,_,_ = chi2_contingency([[freq_c1, freq_c2], 
                  [freq_c1_other, freq_c2_other]],
                  lambda_='log-likelihood') # this specifies using the LL measure
print("Log-likelihood: ", llr)
print("p-value: ", p_value)

Log-likelihood:  4.116294033801272
p-value:  0.04247202129760274


In [94]:
tokenized_text_corpus_1 = BOW2
tokenized_text_corpus_2 = sarisfied_list

counts_c1 = Counter(tokenized_text_corpus_1)
counts_c2 = Counter(tokenized_text_corpus_2)

freq_c1_other = sum(counts_c1.values()) 
freq_c2_other = sum(counts_c2.values())

In [95]:
llr, p_value,_,_ = chi2_contingency([[freq_c1, freq_c2], 
                  [freq_c1_other, freq_c2_other]],
                  lambda_='log-likelihood') # this specifies using the LL measure
print("Log-likelihood: ", llr)
print("p-value: ", p_value)

Log-likelihood:  4.505111852689309
p-value:  0.03379368592884378


From the results its more likely to have the satisfied words in BOW2 but before precenting my answer i should have done normalization.One subset was way smaller than the other,but due to restricted time i was not able to present that.

## Conclusion

Different from i have done so far,my analaysis was in a real case scenario,i used the dataset to create my insites and finish the assigment with a solid result.My answer to the producer:"Data are the most important fuel in order to build a proper analysis,my method proves the linear model to be more effective than the netflix one.So i strongly recomend you follow the result and release it Linear!"